In [ ]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [53.1 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:

!wget -O veh -q https://data.cityofnewyork.us/api/views/bm4k-52h4/rows.csv?accessType=DOWNLOAD


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -y ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-05-01 14:23:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.3.7, 18.205.129.29, 35.170.10.11, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.3.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13828408 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  53.6MB/s    in 0.2s    

2021-05-01 14:23:32 (53.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13828408/13828408]

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compr

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Collision")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
col=spark.read.option("header",True).csv("./veh",inferSchema=True);

In [ ]:
col.schema

StructType(List(StructField(UNIQUE_ID,IntegerType,true),StructField(COLLISION_ID,IntegerType,true),StructField(CRASH_DATE,StringType,true),StructField(CRASH_TIME,StringType,true),StructField(VEHICLE_ID,StringType,true),StructField(STATE_REGISTRATION,StringType,true),StructField(VEHICLE_TYPE,StringType,true),StructField(VEHICLE_MAKE,StringType,true),StructField(VEHICLE_MODEL,StringType,true),StructField(VEHICLE_YEAR,IntegerType,true),StructField(TRAVEL_DIRECTION,StringType,true),StructField(VEHICLE_OCCUPANTS,IntegerType,true),StructField(DRIVER_SEX,StringType,true),StructField(DRIVER_LICENSE_STATUS,StringType,true),StructField(DRIVER_LICENSE_JURISDICTION,StringType,true),StructField(PRE_CRASH,StringType,true),StructField(POINT_OF_IMPACT,StringType,true),StructField(VEHICLE_DAMAGE,StringType,true),StructField(VEHICLE_DAMAGE_1,StringType,true),StructField(VEHICLE_DAMAGE_2,StringType,true),StructField(VEHICLE_DAMAGE_3,StringType,true),StructField(PUBLIC_PROPERTY_DAMAGE,StringType,true),Str

In [ ]:
col.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: string (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- VEHICLE_ID: string (nullable = true)
 |-- STATE_REGISTRATION: string (nullable = true)
 |-- VEHICLE_TYPE: string (nullable = true)
 |-- VEHICLE_MAKE: string (nullable = true)
 |-- VEHICLE_MODEL: string (nullable = true)
 |-- VEHICLE_YEAR: integer (nullable = true)
 |-- TRAVEL_DIRECTION: string (nullable = true)
 |-- VEHICLE_OCCUPANTS: integer (nullable = true)
 |-- DRIVER_SEX: string (nullable = true)
 |-- DRIVER_LICENSE_STATUS: string (nullable = true)
 |-- DRIVER_LICENSE_JURISDICTION: string (nullable = true)
 |-- PRE_CRASH: string (nullable = true)
 |-- POINT_OF_IMPACT: string (nullable = true)
 |-- VEHICLE_DAMAGE: string (nullable = true)
 |-- VEHICLE_DAMAGE_1: string (nullable = true)
 |-- VEHICLE_DAMAGE_2: string (nullable = true)
 |-- VEHICLE_DAMAGE_3: string (nullable = true)
 |-- PUBLIC_PROPERTY_DAM

In [ ]:
col.select("CRASH_TIME").show(10)


+----------+
|CRASH_TIME|
+----------+
|      9:03|
|      8:15|
|     17:18|
|     20:34|
|     21:15|
|     17:35|
|     13:15|
|     12:39|
|     11:20|
|     18:12|
+----------+
only showing top 10 rows



In [ ]:
## Converting CRASH TIME type: String to timestamp 
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_timestamp

col= col.withColumn('CRASH_TIME',to_timestamp(col["CRASH_TIME"], 'H:mm'))


In [ ]:
col.select("CRASH_TIME").show(10)

+-------------------+
|         CRASH_TIME|
+-------------------+
|1970-01-01 09:03:00|
|1970-01-01 08:15:00|
|1970-01-01 17:18:00|
|1970-01-01 20:34:00|
|1970-01-01 21:15:00|
|1970-01-01 17:35:00|
|1970-01-01 13:15:00|
|1970-01-01 12:39:00|
|1970-01-01 11:20:00|
|1970-01-01 18:12:00|
+-------------------+
only showing top 10 rows



In [ ]:

### Removing YYYY-MM-DD from CRASH TIME
from pyspark.sql.types import DateType
from pyspark.sql.functions import date_format
from pyspark.sql import functions as f

col = col.withColumn("CRASH_TIME", f.date_format("CRASH_TIME", 'HH:mm:ss'))

In [ ]:
#Change CRASH DATE type from String to Date
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date

#col.withColumn("CRASH DATE", col['CRASH DATE'].cast(DateType())).show()

 
col= col.withColumn('CRASH_DATE',to_date(col["CRASH_DATE"], 'MM/dd/yyyy'))
col.select("CRASH_DATE").show(10)

+----------+
|CRASH_DATE|
+----------+
|2012-09-07|
|2019-09-23|
|2015-10-02|
|2015-10-04|
|2013-04-25|
|2016-05-02|
|2019-10-24|
|2016-08-18|
|2013-07-16|
|2012-11-26|
+----------+
only showing top 10 rows



In [ ]:
col.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- VEHICLE_ID: string (nullable = true)
 |-- STATE_REGISTRATION: string (nullable = true)
 |-- VEHICLE_TYPE: string (nullable = true)
 |-- VEHICLE_MAKE: string (nullable = true)
 |-- VEHICLE_MODEL: string (nullable = true)
 |-- VEHICLE_YEAR: integer (nullable = true)
 |-- TRAVEL_DIRECTION: string (nullable = true)
 |-- VEHICLE_OCCUPANTS: integer (nullable = true)
 |-- DRIVER_SEX: string (nullable = true)
 |-- DRIVER_LICENSE_STATUS: string (nullable = true)
 |-- DRIVER_LICENSE_JURISDICTION: string (nullable = true)
 |-- PRE_CRASH: string (nullable = true)
 |-- POINT_OF_IMPACT: string (nullable = true)
 |-- VEHICLE_DAMAGE: string (nullable = true)
 |-- VEHICLE_DAMAGE_1: string (nullable = true)
 |-- VEHICLE_DAMAGE_2: string (nullable = true)
 |-- VEHICLE_DAMAGE_3: string (nullable = true)
 |-- PUBLIC_PROPERTY_DAMAG

In [ ]:
print(col.select("VEHICLE_DAMAGE_1","VEHICLE_DAMAGE_2","VEHICLE_DAMAGE_3").distinct().show(30));
## Checking if null value is present

+--------------------+--------------------+--------------------+
|    VEHICLE_DAMAGE_1|    VEHICLE_DAMAGE_2|    VEHICLE_DAMAGE_3|
+--------------------+--------------------+--------------------+
|    Right Side Doors|Right Rear Quarte...|               Other|
|Left Front Quarte...|Left Rear Quarter...|           No Damage|
|     Left Side Doors|   Left Front Bumper|Right Front Quart...|
|   Left Front Bumper|  Right Front Bumper|Right Rear Quarte...|
|   Left Front Bumper|    Center Front End|       Undercarriage|
|   Left Front Bumper|    Center Front End|     Left Side Doors|
|Right Rear Quarte...|       Undercarriage|           No Damage|
|     Center Back End|   Right Rear Bumper|     Left Side Doors|
|    Right Side Doors|   Left Front Bumper|    Left Rear Bumper|
|     Center Back End|   Left Front Bumper|                Roof|
|   Left Front Bumper|Right Rear Quarte...|           No Damage|
|  Right Front Bumper|Left Front Quarte...|                Roof|
|           No Damage|   

In [ ]:
print(col.select("DRIVER_LICENSE_STATUS").distinct().show(30));
## Checking if null value is present

+---------------------+
|DRIVER_LICENSE_STATUS|
+---------------------+
|                 null|
|           Unlicensed|
|               Permit|
|             Licensed|
+---------------------+

None


In [ ]:

## Setting all null values to Unknown 
col=col.fillna(value="Unknown",subset=["DRIVER_LICENSE_STATUS"])

In [ ]:
print(col.select("DRIVER_LICENSE_JURISDICTION").distinct().show(30));
## Checking if null value is present

+---------------------------+
|DRIVER_LICENSE_JURISDICTION|
+---------------------------+
|                   A,NEJADE|
|                         SC|
|                         AZ|
|                        ZZZ|
|                         NS|
|                         LA|
|                         MN|
|                         NJ|
|                         MX|
|                         DC|
|                         OR|
|                         NT|
|                       null|
|                         VA|
|                        NB1|
|                         RI|
|                         KY|
|                         WY|
|                         BC|
|                         NH|
|                         MI|
|                         NV|
|                         WI|
|                         ID|
|                         CA|
|                         CT|
|                         NE|
|                         MT|
|                         NC|
|                         PQ|
+---------

In [ ]:
## Setting all null values to Unknown 
col=col.fillna(value="Not Provided",subset=["DRIVER_LICENSE_JURISDICTION"])

In [ ]:
print(col.select("POINT_OF_IMPACT").distinct().show(30));
## Checking if null value is present

+--------------------+
|     POINT_OF_IMPACT|
+--------------------+
|Left Rear Quarter...|
|     Center Back End|
|Left Front Quarte...|
|                null|
|  Right Front Bumper|
|    Left Rear Bumper|
|    Right Side Doors|
|                Roof|
|Right Front Quart...|
|               Other|
|          Overturned|
|             Trailer|
|Right Rear Quarte...|
|     Left Side Doors|
|   Left Front Bumper|
|          Demolished|
|    Center Front End|
|       Undercarriage|
|   Right Rear Bumper|
|           No Damage|
+--------------------+

None


In [ ]:
## Setting all null values to Unknown 
col=col.fillna(value="Not Provided",subset=["POINT_OF_IMPACT"])

In [ ]:
print(col.select("VEHICLE_TYPE","VEHICLE_MAKE","VEHICLE_MODEL").distinct().show(30));
## Checking if null value is present

+--------------------+----------------+-------------+
|        VEHICLE_TYPE|    VEHICLE_MAKE|VEHICLE_MODEL|
+--------------------+----------------+-------------+
|Tractor Truck Diesel|           DORSE|         null|
|Station Wagon/Spo...|   JEEP -CAR/SUV|     JEEP GCH|
|           Box Truck|  INTL-TRUCK/BUS|         4400|
|Station Wagon/Spo...|   JEEP -CAR/SUV|     JEEP PAT|
|               Sedan|           WAGON|         null|
|          4 dr sedan|   ACUR -CAR/SUV|          ATL|
|           Box Truck|           WORKH|         null|
|          Motorcycle|       HOND -MCL|          CTX|
|                null|         UNKNOWN|      UNKNOWN|
|Tractor Truck Gas...|  FRHT-TRUCK/BUS|      FRHT TK|
|               trail|            null|         null|
|               Sedan|   HUMM -CAR/SUV|         null|
|Station Wagon/Spo...|           HONDA|         null|
|                 Van|             unk|         null|
|       Pick-up Truck|             ihc|         null|
|                 Bus|      

In [ ]:
## Setting all null values to Unknown 
col=col.fillna(value="Not Provided",subset=["VEHICLE_TYPE","VEHICLE_MAKE","VEHICLE_MODEL"])

In [ ]:
print(col.select("STATE_REGISTRATION").distinct().show(100));
## Checking if null value is present

+------------------+
|STATE_REGISTRATION|
+------------------+
|                SC|
|                AZ|
|                UA|
|                NS|
|                LA|
|                MN|
|                NJ|
|                MX|
|                DC|
|                ZZ|
|                OR|
|                NT|
|              null|
|                VA|
|                RI|
|                WY|
|                KY|
|                BC|
|                NH|
|                LR|
|                MI|
|                NV|
|                GV|
|                QB|
|                WI|
|                ID|
|                CA|
|                CT|
|                NE|
|                MT|
|                NC|
|                VT|
|                PQ|
|                MD|
|                DE|
|                MO|
|                VI|
|                IL|
|                ME|
|                MB|
|                GU|
|                WA|
|                ND|
|                MS|
|            

In [ ]:
print(col.select("TRAVEL_DIRECTION").distinct().show(20));
## Checking if null value is present

+----------------+
|TRAVEL_DIRECTION|
+----------------+
|            null|
|               E|
|       Northwest|
|               U|
|         Unknown|
|           South|
|       Southeast|
|               -|
|               N|
|               W|
|               S|
|            East|
|       Southwest|
|            West|
|           North|
|       Northeast|
+----------------+

None


In [ ]:
## Setting all null values to Unknown 
col=col.fillna(value="unknown",subset=["TRAVEL_DIRECTION"])

In [ ]:
print(col.select("VEHICLE_OCCUPANTS").distinct().show(20));

+-----------------+
|VEHICLE_OCCUPANTS|
+-----------------+
|              148|
|              392|
|               31|
|          9999999|
|               65|
|               53|
|               34|
|              211|
|               81|
|               28|
|              183|
|        999999999|
|               76|
|               26|
|               27|
|               44|
|              743|
|               12|
|               22|
|             2013|
+-----------------+
only showing top 20 rows

None


In [ ]:
print(col.select("DRIVER_SEX").distinct().show(20));

+----------+
|DRIVER_SEX|
+----------+
|         F|
|      null|
|         M|
|         U|
+----------+

None


In [ ]:
col=col.fillna(value="Not Provided",subset=["DRIVER_SEX"])

In [ ]:
print(col.select("DRIVER_LICENSE_STATUS").distinct().show(20));

+---------------------+
|DRIVER_LICENSE_STATUS|
+---------------------+
|              Unknown|
|           Unlicensed|
|               Permit|
|             Licensed|
+---------------------+

None


In [ ]:
print(col.select("DRIVER_LICENSE_JURISDICTION").distinct().show(20));

+---------------------------+
|DRIVER_LICENSE_JURISDICTION|
+---------------------------+
|                   A,NEJADE|
|                         SC|
|                         AZ|
|                        ZZZ|
|                         NS|
|                         LA|
|                         MN|
|                         NJ|
|                         MX|
|                         DC|
|                         OR|
|                         NT|
|                         VA|
|                        NB1|
|                         RI|
|                         KY|
|                         WY|
|                         BC|
|                         NH|
|                         MI|
+---------------------------+
only showing top 20 rows

None


In [ ]:
print(col.select("PRE_CRASH").distinct().show(20));

+--------------------+
|           PRE_CRASH|
+--------------------+
|Making Right Turn...|
|             Merging|
| Starting in Traffic|
|             Passing|
|       Making U Turn|
|Starting from Par...|
|  Stopped in Traffic|
|              Other*|
|             Backing|
|Making Left Turn ...|
|    Making Left Turn|
|Entering Parked P...|
|Going Straight Ahead|
|      Police Pursuit|
|      Changing Lanes|
| Slowing or Stopping|
|Avoiding Object i...|
|        Not Provided|
|              Parked|
|   Making Right Turn|
+--------------------+

None


In [ ]:
col=col.fillna(value="Not Provided",subset=["PRE_CRASH"])

In [ ]:
print(col.select("POINT_OF_IMPACT").distinct().show(30));

+--------------------+
|     POINT_OF_IMPACT|
+--------------------+
|Left Rear Quarter...|
|     Center Back End|
|Left Front Quarte...|
|  Right Front Bumper|
|    Left Rear Bumper|
|    Right Side Doors|
|                Roof|
|Right Front Quart...|
|               Other|
|          Overturned|
|             Trailer|
|Right Rear Quarte...|
|     Left Side Doors|
|   Left Front Bumper|
|          Demolished|
|    Center Front End|
|       Undercarriage|
|   Right Rear Bumper|
|        Not Provided|
|           No Damage|
+--------------------+

None


In [ ]:
print(col.select("PUBLIC_PROPERTY_DAMAGE","PUBLIC_PROPERTY_DAMAGE_TYPE").distinct().show(30));

+----------------------+---------------------------+
|PUBLIC_PROPERTY_DAMAGE|PUBLIC_PROPERTY_DAMAGE_TYPE|
+----------------------+---------------------------+
|           Unspecified|       FENCE OF 503 CARY...|
|           Unspecified|       A NO STANDING TRA...|
|           Unspecified|        UTILITY LIGHT POST.|
|           Unspecified|       DAMAGED TREE GUAR...|
|           Unspecified|       TRAFFIC CONTROL D...|
|           Unspecified|              NYPD VMS SIGN|
|           Unspecified|               HOUSE GARAGE|
|           Unspecified|       DENTS AND SCRATCH...|
|           Unspecified|       FENCE OF 853 EAST...|
|           Unspecified|       TRAFFIC SIGNAL DE...|
|           Unspecified|       BRICK WALL 2FT X 8FT|
|           Unspecified|       NOTIFIED DEPARTME...|
|           Unspecified|       METAL GATE IN FRO...|
|           Unspecified|       FRONT GATE TO ENT...|
|           Unspecified|       DAMAGE TO BUILDIN...|
|           Unspecified|       LIGHT POLE ON T

In [ ]:
col=col.fillna(value="Not Provided",subset=["PUBLIC_PROPERTY_DAMAGE"])

In [ ]:
col.printSchema()

root
 |-- UNIQUE_ID: integer (nullable = true)
 |-- COLLISION_ID: integer (nullable = true)
 |-- CRASH_DATE: date (nullable = true)
 |-- CRASH_TIME: string (nullable = true)
 |-- VEHICLE_ID: string (nullable = true)
 |-- STATE_REGISTRATION: string (nullable = true)
 |-- VEHICLE_TYPE: string (nullable = true)
 |-- VEHICLE_MAKE: string (nullable = true)
 |-- VEHICLE_MODEL: string (nullable = true)
 |-- VEHICLE_YEAR: integer (nullable = true)
 |-- TRAVEL_DIRECTION: string (nullable = true)
 |-- VEHICLE_OCCUPANTS: integer (nullable = true)
 |-- DRIVER_SEX: string (nullable = true)
 |-- DRIVER_LICENSE_STATUS: string (nullable = false)
 |-- DRIVER_LICENSE_JURISDICTION: string (nullable = false)
 |-- PRE_CRASH: string (nullable = true)
 |-- POINT_OF_IMPACT: string (nullable = false)
 |-- VEHICLE_DAMAGE: string (nullable = true)
 |-- VEHICLE_DAMAGE_1: string (nullable = true)
 |-- VEHICLE_DAMAGE_2: string (nullable = true)
 |-- VEHICLE_DAMAGE_3: string (nullable = true)
 |-- PUBLIC_PROPERTY_DA